In [24]:
#unique_age_milestones = df['Dimension'].unique()


In [25]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [26]:
df = pd.read_csv("Datasets/Vaccination_Coverage_among_Young_Children__0-35_Months__20241101.csv")

In [27]:
# Define the state-to-region mapping for the 9 census regions
state_to_region = {
    # Middle Atlantic
    'New York': 'Middle Atlantic', 'NY-Rest of state': 'Middle Atlantic', 'NY-City of New York': 'Middle Atlantic', 
    'New Jersey': 'Middle Atlantic', 'Pennsylvania': 'Middle Atlantic', 
    'PA-Philadelphia': 'Middle Atlantic', 'PA-Rest of state': 'Middle Atlantic',

    # New England
    'Vermont': 'New England', 'New Hampshire': 'New England', 'Massachusetts': 'New England', 
    'Connecticut': 'New England', 'Rhode Island': 'New England', 'Maine': 'New England',

    # East North Central
    'Indiana': 'East North Central', 'Illinois': 'East North Central', 'Michigan': 'East North Central', 
    'Ohio': 'East North Central', 'Wisconsin': 'East North Central', 
    'IL-City of Chicago': 'East North Central', 'IL-Rest of state': 'East North Central',

    # West North Central
    'Iowa': 'West North Central', 'Kansas': 'West North Central', 'Minnesota': 'West North Central',
    'Missouri': 'West North Central', 'Nebraska': 'West North Central', 
    'North Dakota': 'West North Central', 'South Dakota': 'West North Central',

    # South Atlantic
    'Delaware': 'South Atlantic', 'District of Columbia': 'South Atlantic', 'Florida': 'South Atlantic', 
    'Georgia': 'South Atlantic', 'Maryland': 'South Atlantic', 'North Carolina': 'South Atlantic', 
    'South Carolina': 'South Atlantic', 'Virginia': 'South Atlantic', 'West Virginia': 'South Atlantic',

    # East South Central
    'Alabama': 'East South Central', 'Kentucky': 'East South Central', 
    'Mississippi': 'East South Central', 'Tennessee': 'East South Central', 

    # West South Central
    'Arkansas': 'West South Central', 'Louisiana': 'West South Central', 'Oklahoma': 'West South Central', 
    'Texas': 'West South Central', 'TX-City of Houston': 'West South Central', 
    'TX-Rest of state': 'West South Central', 'TX-Dallas County': 'West South Central', 
    'TX-Bexar County': 'West South Central', 'TX-El Paso County': 'West South Central', 
    'TX-Hidalgo County': 'West South Central', 'TX-Tarrant County': 'West South Central',

    # Mountain
    'Arizona': 'Mountain', 'Colorado': 'Mountain', 'Idaho': 'Mountain', 
    'New Mexico': 'Mountain', 'Montana': 'Mountain', 'Utah': 'Mountain', 
    'Nevada': 'Mountain', 'Wyoming': 'Mountain',

    # Pacific
    'Alaska': 'Pacific', 'California': 'Pacific', 'Hawaii': 'Pacific', 
    'Oregon': 'Pacific', 'Washington': 'Pacific'
}


In [28]:
df = df[df['Vaccine'] == 'DTaP']

unique_vaccines = df['Vaccine'].unique()
print("Unique values in 'Vaccine' column after filtering:", unique_vaccines)

Unique values in 'Vaccine' column after filtering: ['DTaP']


In [29]:
# Filter to include only rows where 'Dimension Type' is 'Age'
df_age_only = df[df['Dimension Type'] == 'Age']

# Map Census Region and handle any unmapped entries as 'Unknown'
df_age_only['Census Region'] = df_age_only['Geography'].map(state_to_region).fillna('Unknown')

# Filter out rows with 'Unknown' regions if they exist
df_age_only = df_age_only[df_age_only['Census Region'] != 'Unknown']

# Extract the age in months from the 'Dimension' column
df_age_only['Age Milestone (months)'] = df_age_only['Dimension'].str.extract(r'(\d+)').astype(float)

# Drop any rows with missing values in relevant columns
df_age_only = df_age_only.dropna(subset=['Age Milestone (months)', 'Census Region', 'Estimate (%)'])

# Convert Estimate (%) to numeric if not already
df_age_only['Estimate (%)'] = pd.to_numeric(df_age_only['Estimate (%)'], errors='coerce')
df_age_only = df_age_only.dropna(subset=['Estimate (%)'])



C:\Users\16692\AppData\Local\Temp\ipykernel_6948\1665058878.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_only['Census Region'] = df_age_only['Geography'].map(state_to_region).fillna('Unknown')


In [30]:
# Run Two-Way ANOVA for Region and Age with interaction
model = ols('Q("Estimate (%)") ~ Q("Census Region") + Q("Age Milestone (months)") + Q("Census Region"):Q("Age Milestone (months)")', data=df_age_only).fit()
anova_results = anova_lm(model, typ=2)

# Display ANOVA table for interpretation
print("\nANOVA Results:\n", anova_results)

# Check if p-value for Census Region is significant (< 0.01) to justify running Tukey's test
if anova_results.loc['Q("Census Region")', 'PR(>F)'] < 0.01:
    tukey = pairwise_tukeyhsd(endog=df_age_only['Estimate (%)'], groups=df_age_only['Census Region'], alpha=0.01)
    print("\nTukey HSD Test Results:\n", tukey)

# Additional output for F and p-values for easier reference
print("\nF and p-values for each effect in the ANOVA:")
for effect in ['Q("Census Region")', 'Q("Age Milestone (months)")', 'Q("Census Region"):Q("Age Milestone (months)")']:
    f_value = anova_results.loc[effect, 'F']
    p_value = anova_results.loc[effect, 'PR(>F)']
    print(f"{effect} - F-value: {f_value}, p-value: {p_value}")


ANOVA Results:
                                                        sum_sq       df  \
Q("Census Region")                               42125.593352      8.0   
Q("Age Milestone (months)")                     140213.326617      1.0   
Q("Census Region"):Q("Age Milestone (months)")    2240.135099      8.0   
Residual                                        825053.726496  11596.0   

                                                          F         PR(>F)  
Q("Census Region")                                74.008571  1.528791e-119  
Q("Age Milestone (months)")                     1970.676191   0.000000e+00  
Q("Census Region"):Q("Age Milestone (months)")     3.935593   1.168006e-04  
Residual                                                NaN            NaN  

Tukey HSD Test Results:
             Multiple Comparison of Means - Tukey HSD, FWER=0.01             
      group1             group2       meandiff p-adj   lower   upper  reject
-----------------------------------------------

In [31]:
# Check unique values in 'Dimension Type' to confirm only 'Age' is present
unique_dimension_types = df_age_only['Dimension Type'].unique()
print("Unique values in 'Dimension Type':", unique_dimension_types)


Unique values in 'Dimension Type': ['Age']


In [32]:
# Check unique values in 'Dimension' to ensure they only contain age milestones
unique_dimensions = df_age_only['Dimension'].unique()
print("Unique values in 'Dimension':", unique_dimensions)


Unique values in 'Dimension': ['19 Months' '5 Months' '13 Months' '35 Months' '24 Months' '7 Months'
 '3 Months']


In [39]:
# Display a sample of rows to verify the filtering
df_age_only[['Dimension Type', 'Dimension']].sample(50)



,Dimension Type,Dimension
5864,Age,24 Months
22476,Age,7 Months
89952,Age,5 Months
116930,Age,24 Months
39495,Age,35 Months
90843,Age,13 Months
74034,Age,13 Months
124979,Age,19 Months
78326,Age,5 Months
71089,Age,19 Months


In [35]:
df_age_only.sample(50)

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size,Census Region,Age Milestone (months)
117618,DTaP,≥3 Doses,States/Local Areas,IL-Rest of state,2012,Age,13 Months,87.4,84.5 to 89.8,330.0,East North Central,13.0
94019,DTaP,≥4 Doses,States/Local Areas,Arkansas,2014,Age,19 Months,62.8,54.7 to 70.2,239.0,West South Central,19.0
69190,DTaP,≥2 Doses,States/Local Areas,New York,2019-2020,Age,5 Months,78.6,75.0 to 81.8,1261.0,Middle Atlantic,5.0
122030,DTaP,≥3 Doses,States/Local Areas,PA-Rest of state,2015,Age,13 Months,88.0,79.4 to 93.3,299.0,Middle Atlantic,13.0
7081,DTaP,≥4 Doses,States/Local Areas,Kansas,2019,Age,19 Months,71.0,63.8 to 77.2,358.0,West North Central,19.0
96673,DTaP,≥4 Doses,States/Local Areas,TX-Rest of state,2014-2015,Age,24 Months,79.2,76.2 to 82.1,1397.0,West South Central,24.0
37442,DTaP,≥3 Doses,States/Local Areas,Louisiana,2015-2016,Age,13 Months,88.0,83.1 to 91.6,540.0,West South Central,13.0
52981,DTaP,≥1 Dose,States/Local Areas,NY-City of New York,2021,Age,3 Months,81.5,72.2 to 88.1,218.0,Middle Atlantic,3.0
22530,DTaP,≥3 Doses,States/Local Areas,Oklahoma,2015-2016,Age,7 Months,64.3,59.4 to 68.9,594.0,West South Central,7.0
18382,DTaP,≥3 Doses,States/Local Areas,Wisconsin,2016-2017,Age,13 Months,92.7,89.0 to 95.2,513.0,East North Central,13.0
